## H2O Linear Model

In [ ]:
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch

In [ ]:
h2o.cluster().shutdown()

In [ ]:
from pyspark.sql import SparkSession
from pysparkling import *
import h2o



# Create Spark session
spark = SparkSession.builder.appName("YourAppName").getOrCreate()

h2o.init()
# Initialize H2OContext
hc = H2OContext.getOrCreate()

# Get H2OConf from H2OContext
h2o_conf = hc.getConf()

# Set H2OConf properties
h2o_conf.set("spark.ext.h2o.client.language", "python")


In [ ]:
h2o_df = h2o.import_file(file_path)
# h2o_df = spark.read.option("header", True).csv(file_path)
h2o_df = h2o_df.drop("userId")

try:
    h2o_df.drop('C1')
    h2o_df.drop('userId')
except Exception as e:
    pass

col_features = h2o_df.columns
col_features.remove("label")
# Split the data into training and validation sets
# Split the data into training and validation sets
train, valid = h2o_df.split_frame(ratios=[0.85])

# Define predictor and response columns
predictor_cols = col_features
response_col = "label"




In [ ]:


# Define hyperparameter grid
hyperparams = {
    'alpha': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0],  # Regularization parameter
    'lambda': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0],
}

# Initialize the linear regression model
linear_reg = H2OGeneralizedLinearEstimator(family='binomial', link="logit")

# Perform grid search
grid_search = H2OGridSearch(linear_reg, hyperparams)
grid_search.train(x=predictor_cols, y=response_col, training_frame=train)

# Get the best model from the grid search
best_model = grid_search.get_grid()[0]

# Make predictions on the validation set
preds = best_model.predict(valid)

# Get model performance on the validation set
validation_performance = best_model.model_performance(valid)


In [ ]:

print(validation_performance.F1())
print(validation_performance.F2())

In [ ]:
validation_performance

In [ ]:



h2o_df = h2o.import_file(file_path)
# h2o_df = spark.read.option("header", True).csv(file_path)
h2o_df = h2o_df.drop("userId")

try:
    h2o_df.drop('C1')
    h2o_df.drop('userId')
except Exception as e:
    pass

col_features = h2o_df.columns
col_features.remove("label")
# Split the data into training and validation sets
# Split the data into training and validation sets
train, valid = h2o_df.split_frame(ratios=[0.85])

# Define predictor and response columns
predictor_cols = col_features
response_col = "label"


# Define hyperparameter grid
hyperparams = {
    'learn_rate': [0.01, 0.1, 0.2],
    'ntrees': [50, 100, 200],
    'max_depth': [3, 4, 5],
}

gradient_boost = H2OGradientBoostingEstimator(seed=42)


# Perform grid search
grid_search = H2OGridSearch(gradient_boost, hyperparams)
grid_search.train(x=predictor_cols, y=response_col, training_frame=train)

# Get the best model from the grid search
best_model = grid_search.get_grid()[0]

# Make predictions on the validation set
preds = best_model.predict(valid)

# Get model performance on the validation set
validation_performance = best_model.model_performance(valid)



In [ ]:

validation_performance